###Exercise 8-1

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torch import nn, from_numpy, tensor, optim, cuda
from torchvision import datasets, transforms 
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time


In [ ]:
class TitanicDataset(Dataset):
    """ Titanic dataset."""

    # Initialize your data, download, etc.
    def __init__(self):
        df = pd.read_csv('./drive/My Drive/titanic_data/train.csv')
        cols = ['Name', 'Ticket', 'Cabin']
        df = df.drop(cols, axis=1)
        df = df.dropna()
        dummies = []
        cols = ['Pclass', 'Sex', 'Embarked']
        for col in cols:
            dummies.append(pd.get_dummies(df[col]))
        titanic_dummies = pd.concat(dummies, axis=1)
        df = pd.concat((df,titanic_dummies), axis=1)
        df = df.drop(['Pclass', 'Sex', 'Embarked'], axis=1)
        df['Age'] = df['Age'].interpolate()
        x_data = df.values
        self.x_data = np.delete(x_data, 1, axis=1)
        self.y_data = df['Survived'].values
        self.len = x_data.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
class Model(nn.Module):

    def __init__(self):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        self.l0 = nn.Linear(32,13)
        self.l1 = nn.Linear(13, 6)
        self.l2 = nn.Linear(6, 4)
        self.l3 = nn.Linear(4, 1)

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.sigmoid(self.l1(x))
        out2 = self.sigmoid(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred


# our model
model = Model()

In [ ]:
dataset = TitanicDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=2)


criterion = nn.BCELoss(reduction='sum')
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training loop
for epoch in range(2):
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data

        # inputs=np.float32(inputs)
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(inputs.float())

        # Compute and print loss
        loss = criterion(y_pred, labels.float())
        print(f'Epoch {epoch + 1} | Batch: {i+1} | Loss: {loss.item():.4f}')

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



###Exercise 9-1,2
CrossEntropyLoss VS Negative Log-LikelihoodLoss


*Any loss consisting of a negative log-likelihood is a cross entropy between the empirical distribution defined by the training set and the probability distribution defined by model.

In [ ]:
class OttoDataset(Dataset):
    # Initialize your data, download, etc.
    def __init__(self):
        xy = np.loadtxt('./drive/My Drive/otto_data/train.csv',
                        delimiter=',', dtype=np.float32, skiprows=1)
        self.len = xy.shape[0]
        self.x_data = from_numpy(xy[:, 1:-1])
        self.y_data = from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
batch_size = 32
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training otto Model on {device}\n{"=" * 44}')

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l2 = nn.Linear(416, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 5)

    def forward(self, x):
        x = x.view(-1, 416)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l2(x.float()))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        print(len(output))
        print(len(target))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')
